In [1]:
import pandas as pd

import os
from Bio import SeqIO


from collections import defaultdict


In [2]:
filename = "/home/zengs/Code/signalp-6.0/data/train_set.fasta"

dataset_threshold = 10
small_dataset_set_filename = f"/home/zengs/Code/signalp-6.0/data/small_data/small_dataset_{dataset_threshold}.fasta"



In [3]:
!head -n 5 $filename

>Q8TF40|EUKARYA|NO_SP|0
MAPTLFQKLFSKRTGLGAPGRDARDPDCGFSWPLPEFDPSQIRLIVYQDCERRGRNVLFDSSVKRRNEDI
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
>Q1ENB6|EUKARYA|NO_SP|1
MDFTSLETTTFEEVVIALGSNVGNRMNNFKEALRLMKDYGISVTRHSCLYETEPVHVTDQPRFLNAAIRG


In [4]:
sequences = []
for seq_record in SeqIO.parse(filename, "fasta"):
    sequences.append(seq_record)
    
    
annotation_dict = defaultdict(list)
for sequence in sequences:
    annotation_dict["id"].append(sequence.id)
    # annotation_dict["seq"].append(sequence.seq)
    
    annotation_dict["uniprot"].append(sequence.id.split("|")[0])
    annotation_dict["kingdom"].append(sequence.id.split("|")[1])
    annotation_dict["type"].append(sequence.id.split("|")[2])
    annotation_dict["partition"].append(sequence.id.split("|")[3])
    annotation_dict["sequence"].append(sequence.id.split("|")[3])
  
    seq_len = len(sequence.seq) // 2
    annotation_dict["aac_seq"].append(str(sequence.seq)[:seq_len])
    annotation_dict["cleavage_seq"].append(str(sequence.seq)[seq_len:])
    


annodatation_df = pd.DataFrame(annotation_dict)
annodatation_df

,id,uniprot,kingdom,type,partition,sequence,aac_seq,cleavage_seq
0,Q8TF40|EUKARYA|NO_SP|0,Q8TF40,EUKARYA,NO_SP,0,0,MAPTLFQKLFSKRTGLGAPGRDARDPDCGFSWPLPEFDPSQIRLIV...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...
1,Q1ENB6|EUKARYA|NO_SP|1,Q1ENB6,EUKARYA,NO_SP,1,1,MDFTSLETTTFEEVVIALGSNVGNRMNNFKEALRLMKDYGISVTRH...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...
2,P36001|EUKARYA|NO_SP|2,P36001,EUKARYA,NO_SP,2,2,MDDISGRQTLPRINRLLEHVGNPQDSLSILHIAGTNGKETVSKFLT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...
3,P55317|EUKARYA|NO_SP|0,P55317,EUKARYA,NO_SP,0,0,MLGTVKMEGHETSDWNSYYADTQEAYSSVPVSNMNSGLGSMNSMNT...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...
4,P35583|EUKARYA|NO_SP|0,P35583,EUKARYA,NO_SP,0,0,MLGAVKMEGHEPSDWSSYYAEPEGYSSVSNMNAGLGMNGMNTYMSM...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...
...,...,...,...,...,...,...,...,...
20285,P0AB09|NEGATIVE|TAT|0,P0AB09,NEGATIVE,TAT,0,0,MDKFDANRRKLLALGGVALGAAILPTPAFATLSTPRPRILTLNNLH...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTOOOOOOOOOOOOOOOO...
20286,P0A5E2|POSITIVE|TAT|2,P0A5E2,POSITIVE,TAT,2,2,MTTMITLRRRFAVAVAGVATAAATTVTLAPAPANAADVYGAIAYSG...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTOOOOOOOOOOO...
20287,P9WQP3|POSITIVE|TAT|1,P9WQP3,POSITIVE,TAT,1,1,MQLVDRVRGAVTGMSRRLVVGAVGAALVSGLVGAVGGTATAGAFSR...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTOOO...
20288,P9WQP2|POSITIVE|TAT|1,P9WQP2,POSITIVE,TAT,1,1,MQLVDRVRGAVTGMSRRLVVGAVGAALVSGLVGAVGGTATAGAFSR...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTOOO...


In [5]:
# "NO_SP": "Other",
# "SP": "Sec/SPI",
# "LIPO": "Sec/SPII",
# "TAT": "Tat/SPI",
# "TATLIPO": "Tat/SPII",
# "PILIN": "Sec/SPIII",

prompt_type_list = ["TATLIPO", "PILIN"]
finetune_df = annodatation_df[~annodatation_df.type.isin(prompt_type_list)]
promt_df = annodatation_df[annodatation_df.type.isin(prompt_type_list)]



In [6]:

data_list = []
for id, row in annodatation_df.groupby(by=["kingdom", "type", "partition"]):
    data_list.append(row.head(dataset_threshold))

small_dataset_df = pd.concat(data_list)

In [7]:
small_dataset_df

,id,uniprot,kingdom,type,partition,sequence,aac_seq,cleavage_seq
13662,P58300|ARCHAEA|LIPO|0,P58300,ARCHAEA,LIPO,0,0,MRRATYAFALLAILVLGVVASGCIGGGTTTPTQTSPATQPTTTQTP...,LLLLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOO...
13668,C7NWW8|ARCHAEA|LIPO|0,C7NWW8,ARCHAEA,LIPO,0,0,MCPRPRRAVLLGLGVAMSAIAGCRETAPSTTQTSDGGPEPTGRSGD...,LLLLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOO...
13672,O30316|ARCHAEA|LIPO|0,O30316,ARCHAEA,LIPO,0,0,MKVLSISLIFFALLLTGCSQVEKIAPDNFNKIFEEGKQFVENTQTE...,LLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
13663,Q58586|ARCHAEA|LIPO|1,Q58586,ARCHAEA,LIPO,1,1,MIKRLIVISILLIVGTVLCGCMEQENVGQQNSEAQEKIVLKIFHAG...,LLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOO...
13666,A4G0R9|ARCHAEA|LIPO|1,A4G0R9,ARCHAEA,LIPO,1,1,MNKKSLNIVVMFGILMILAFSGCVDQNASESTSEDTTPKILKIFHA...,LLLLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOO...
...,...,...,...,...,...,...,...,...
14319,A5U493|POSITIVE|TATLIPO|2,A5U493,POSITIVE,TATLIPO,2,2,MRNRGFGRRELLVAMAMLVSVTGCARHASGARPASTTLPAGADLAD...,TTTTTTTTTTTTTTTTTTTTTTTOOOOOOOOOOOOOOOOOOOOOOO...
15938,P9WKD3|POSITIVE|TATLIPO|2,P9WKD3,POSITIVE,TATLIPO,2,2,MRNRGFGRRELLVAMAMLVSVTGCARHASGARPASTTLPAGADLAD...,TTTTTTTTTTTTTTTTTTTTTTTOOOOOOOOOOOOOOOOOOOOOOO...
15939,P9WKD2|POSITIVE|TATLIPO|2,P9WKD2,POSITIVE,TATLIPO,2,2,MRNRGFGRRELLVAMAMLVSVTGCARHASGARPASTTLPAGADLAD...,TTTTTTTTTTTTTTTTTTTTTTTOOOOOOOOOOOOOOOOOOOOOOO...
15940,P0A5I7|POSITIVE|TATLIPO|2,P0A5I7,POSITIVE,TATLIPO,2,2,MRNRGFGRRELLVAMAMLVSVTGCARHASGARPASTTLPAGADLAD...,TTTTTTTTTTTTTTTTTTTTTTTOOOOOOOOOOOOOOOOOOOOOOO...


In [8]:

## finetine set
with open(small_dataset_set_filename, "w") as output_file:
    for index, row in small_dataset_df.iterrows():
        id = f">{row['id']}\n"
        seq = row["aac_seq"] + "\n" + row["cleavage_seq"]+ "\n"
        
        output_file.write(id)
        output_file.write(seq)
        

In [9]:
!head -n 10 $small_dataset_set_filename

>P58300|ARCHAEA|LIPO|0
MRRATYAFALLAILVLGVVASGCIGGGTTTPTQTSPATQPTTTQTPTQTETQAVECGSGKVVIWHAMQPN
LLLLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
>C7NWW8|ARCHAEA|LIPO|0
MCPRPRRAVLLGLGVAMSAIAGCRETAPSTTQTSDGGPEPTGRSGDTQTDGGGETTRQATVAEGGESATR
LLLLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
>O30316|ARCHAEA|LIPO|0
MKVLSISLIFFALLLTGCSQVEKIAPDNFNKIFEEGKQFVENTQTEIEDIDIEQLIEKAKELGFNTEILT
LLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
>Q58586|ARCHAEA|LIPO|1
